In [1]:
import json

# 读取 txt 文件，创建 entity_id 和数字之间的映射字典
txt_file = '/Data/SA22001084/simKGC/data/WN18RR/entities.txt'
entity_id_map = {}

with open(txt_file, 'r') as f:
    for line in f:
        number, entity_id = line.strip().split()
        entity_id_map[entity_id] = number

# 读取 json 文件
json_file = '/Data/SA22001084/simKGC/data/WN18RR/entities.json'

with open(json_file, 'r') as f:
    json_data = json.load(f)

# 用于存储结果的列表
output_data = []

# 找到 entity 和 entity_id 对应的数字
for entry in json_data:
    entity_id = entry['entity_id']
    corresponding_number = entity_id_map.get(entity_id, None)
    
    if corresponding_number:
        # 将找到的结果添加到输出列表中
        output_data.append({
            'entity': entry['entity'],
            'entity_id': entity_id,
            'corresponding_number': corresponding_number
        })

# 将结果写入新的 JSON 文件
output_file = 'output_entities.json'
with open(output_file, 'w',encoding='utf-8') as f:
    json.dump(output_data, f, ensure_ascii=False,indent=4)

print(f'结果已保存到 {output_file}')

结果已保存到 output_entities.json


In [4]:
#将simkgc的文本文件替换为数字
import json

# 读取 output_entities.json 文件，构建实体名称和对应数字的映射字典
with open('output_entities.json', 'r',encoding='utf-8') as f:
    entity_map = {entry['entity']: entry['corresponding_number'] for entry in json.load(f)}

# 构建关系的映射字典
def load_relation_dict(file_path):
    """
    从指定路径的 txt 文件中读取关系字典。

    参数:
        file_path (str): 包含关系字典的 txt 文件路径。

    返回:
        dict: 关系字典，格式为 {id: relation}。
    """
    relation_dict = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # 去除行末的换行符并按制表符分割
            parts = line.strip().split('\t')
            if len(parts) == 2:  # 确保每行有两部分（id 和 relation）
                relation_id = int(parts[0])  # 将 id 转换为整数
                relation = parts[1]  # 获取关系描述
                relation_dict[relation_id] = relation
    return relation_dict

# 示例调用
file_path = '/Data/SA22001084/simKGC/data/WN18RR/relations.txt'  # 替换为你的文件路径
relation_dict = load_relation_dict(file_path)

# 创建关系反向映射 (从文本到数字)
relation_map = {v: k for k, v in relation_dict.items()}

# 读取需要修改的 JSON 文件
json_file = "/Data/SA22001084/simKGC/checkpoint/wn18rr_final_biobert/wn18rr_final_biobert_valid_raw_bestepoch.json"

with open(json_file, 'r',encoding='utf-8') as f:
    data = json.load(f)

# 替换 head, tail, pred_tail, relation，并处理 topk_score_info
for entry in data:
    # 替换 head, tail, 和 pred_tail
    entry['head'] = entity_map.get(entry['head'], entry['head'])
    entry['tail'] = entity_map.get(entry['tail'], entry['tail'])
    entry['pred_tail'] = entity_map.get(entry['pred_tail'], entry['pred_tail'])

    # 替换 relation
    relation = entry['relation']
    entry['relation'] = relation_map.get(relation, relation)

    # 替换 topk_score_info 中的实体名称
    if 'topk_score_info' in entry:
        try:
            # 解析 topk_score_info 字符串为 JSON
            topk_score_info = json.loads(entry['topk_score_info'])

            # 替换 topk_score_info 中的实体名称为对应的数字
            updated_topk_score_info = {}
            for k, v in topk_score_info.items():
                # 检查是否存在映射，如果没有，记录或者跳过
                if k in entity_map:
                    updated_topk_score_info[entity_map[k]] = v
                else:
                    # 如果找不到映射，可以记录未替换的实体
                    print(f"未找到映射的实体: {k}")
                    updated_topk_score_info[k] = v  # 保留原始键

            # 将更新后的字典重新转换为字符串
            entry['topk_score_info'] = json.dumps(updated_topk_score_info)
        except json.JSONDecodeError:
            print(f"Error decoding topk_score_info for entry: {entry}")

# 将更新后的 JSON 文件保存
output_file = 'biobert_valid_text2id.json'
with open(output_file, 'w',encoding='utf-8') as f:
    json.dump(data, f, indent=4)

print(f'替换后的 JSON 文件已保存为 {output_file}')


替换后的 JSON 文件已保存为 biobert_valid_text2id.json
